# Demo of API usage of the Prox Soft Body Simulator

In [1]:
%matplotlib widget

First we make sure to import all modules that is needed.

In [2]:
import numpy as np
import isl.math.vector3 as V3
import isl.geometry.volume_mesh as VM
import isl.simulators.prox_soft_bodies.api as API
import isl.simulators.prox_soft_bodies.solver as SOLVER
import isl.util.viewer as VIEWER

Second we set up a test scene using a simple beam shape. The beam is created with Dirichlet constraints on its left-most face. We let gravity work on the beam and add some slight damping to the beam as well. The end result should be a beam simulation where we will see the right-most end of the beam bounce up and down.

In [3]:
def wall_sdf(x):
    return x[0] + 2.9

V, T = VM.create_beam(35, 5, 5, 6.0, 1.0, 1.0)

engine = API.Engine()

API.create_soft_body(engine, 'beam', V, T)
API.set_type(engine, 'beam', 'Free')
API.create_dirichlet_conditions(engine, 'beam', wall_sdf)

API.create_material(engine, 'steel')
API.set_elasticity(engine, 'steel', 1.0, 0.3)
API.set_mass_density(engine, 'steel', 1.0)
API.set_constitutive_model(engine, 'steel', API.SVK)
API.set_viscosity(engine, 'steel', 1.0)
API.create_surfaces_interaction(engine,'steel','steel', 0.5)
API.set_gravity(engine, 'beam', (0,0,-10))
API.set_material(engine,'beam','steel')

Next we create a viewer object that will be responsible for showing the animation. We need an auxiliary function that can take contact point data out of the engine and into the viewer.

Once we got this in place then we can create a simulation loop and start the simulation.

In [4]:
viewer = VIEWER.Viewer()
for body in engine.bodies.values():
    opacity = 0.5
    color = V3.make(1.0,0.1,0.1)
    F = VM.to_triangles(body.T)
    viewer.create_mesh(body.name, body.x, F, color, opacity)    
    opacity = 0.8
    color = V3.make(0.1,0.1,1.0)
    viewer.create_mesh(body.name + '0', body.x0, F, color, opacity)
viewer.create_quiver('contacts')


def update_contacts(viewer, engine): 
    K = len(engine.contact_points)
    V = np.zeros((K,3),dtype=np.float64)
    N = np.zeros((K,3),dtype=np.float64)
    for k in range(K):
        V[k,:] = engine.contact_points[k].p
        N[k,:] = engine.contact_points[k].n
    viewer.update_quiver('contacts', V, N, 2.0)
    

def simulation(viewer, engine, monitor=True):
    stats = []
    import time
    dt = engine.params.time_step
    T  = engine.params.total_time
    fps = 1.0/dt
    steps = int(np.round(T*fps))
    
    for i in range(steps):
        for body in engine.bodies.values():
            viewer.update_mesh(body.name, body.x0)
        update_contacts(viewer, engine)
        data = SOLVER.stepper(dt, engine, monitor)
        stats.append(data)
        time.sleep(dt)
    return stats
        
engine.params.total_time = 1.0
viewer.show()
viewer.hide_quiver('contacts')
stats = simulation(viewer, engine, True)

/Users/kenny/opt/anaconda3/envs/cmis/lib/python3.9/site-packages/jupyter_client/session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


Renderer(camera=PerspectiveCamera(aspect=1.25, children=(DirectionalLight(color='white', intensity=0.6, positi…

The stepper function in the SOLVER module is the main work horse of the simulation. It outputs a lot of statistics about the numerics. This data is useful to analyze what happend in the simulation and if the simulator was working or if something unexpected happend.

Often one cares about the convergence of the solver. It is not difficult to extract and plot the convergence rates.

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
ax = plt.subplot(111)
ax.set_title('Converegence rates')
ax.set_xlabel('Iterations')
ax.set_ylabel('Merit')
plt.grid(True)
for i in range(len(stats)):
    data = stats[i]
    if 'residuals' in data.keys():
        residuals = data['residuals']
        reject = data['reject']
        ax.plot( residuals[np.where(reject==False)])
plt.show()

In [5]:
colors = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4',
          '#46f0f0', '#f032e6', '#bcf60c', '#fabebe', '#008080', '#e6beff',
          '#9a6324', '#fffac8', '#800000', '#aaffc3', '#808000', '#ffd8b1',
          '#000075', '#808080', '#ffffff', '#000000']


time_elastic_forces = [ stats[i]['compute_elastic_forces'] for i in range(len(stats)) ]
time_traction_forces = [ stats[i]['compute_traction_forces'] for i in range(len(stats)) ]
time_damping_forces = [ stats[i]['compute_damping_forces'] for i in range(len(stats)) ]
time_external_forces = [ stats[i]['compute_external_forces'] for i in range(len(stats)) ]
time_mass_matrix = [ stats[i]['compute_mass_matrix'] for i in range(len(stats)) ]
time_inverse_mass_matrix = [ stats[i]['compute_inverse_mass_matrix'] for i in range(len(stats)) ]
time_update_bvh = [ stats[i]['update_bvh'] for i in range(len(stats)) ]
time_narrow_phase = [ stats[i]['narrow_phase'] for i in range(len(stats)) ]
time_contact_determination = [ stats[i]['contact_determination'] for i in range(len(stats)) ]
time_contact_point_reduction = [ stats[i]['contact_point_reduction'] for i in range(len(stats)) ]
time_collision_detection = [ stats[i]['collision_detection_time'] for i in range(len(stats)) ]
time_stepper = [ stats[i]['stepper_time'] for i in range(len(stats)) ]
time_kinetic_energy = [ stats[i]['compute_kinetic_energy'] for i in range(len(stats)) ]
time_poential_energy = [ stats[i]['compute_potential_energy'] for i in range(len(stats)) ]
time_elastic_energy = [ stats[i]['compute_elastic_energy'] for i in range(len(stats)) ]

fig = plt.figure()
ax = plt.subplot(111)
ax.set_title('Profiling Timings')
ax.set_xlabel('Step')
ax.set_ylabel('Time [s]')
plt.grid(True)
ax.plot(time_elastic_forces, label='Elastic forces', color=colors[0])
ax.plot(time_traction_forces, label='Traction forces', color=colors[1])
ax.plot(time_damping_forces, label='Damping forces', color=colors[2])
ax.plot(time_external_forces, label='External forces', color=colors[3])
ax.plot(time_mass_matrix, label='Mass matrix', color=colors[4])
ax.plot(time_inverse_mass_matrix, label='Inverse mass', color=colors[5])
ax.plot(time_update_bvh, label='Update bvh', color=colors[6])
ax.plot(time_narrow_phase, label='Narrow phase', color=colors[7])
ax.plot(time_contact_determination, label='Contact determination', color=colors[8])
ax.plot(time_contact_point_reduction, label='Contact reduction', color=colors[9])
ax.plot(time_collision_detection, label='Collision Detection', color=colors[10])
ax.plot(time_stepper, label='Stepper', color=colors[11])
ax.plot(time_kinetic_energy, label='Kinetic', color=colors[12])
ax.plot(time_poential_energy, label='Potential', color=colors[13])
ax.plot(time_elastic_energy, label='Elastic', color=colors[14])
ax.legend()
plt.show()

number_of_overlaps = [ stats[i]['number_of_overlaps'] for i in range(len(stats)) ]
step_sizes = [ stats[i]['dt'] for i in range(len(stats)) ]
number_of_contact_points = [ stats[i]['contact_points'] for i in range(len(stats)) ]
elastic_energy = [ stats[i]['elastic_energy'] for i in range(len(stats)) ]
kinetic_energy = [ stats[i]['elastic_energy'] for i in range(len(stats)) ]
potential_energy = [ stats[i]['elastic_energy'] for i in range(len(stats)) ]
penetrations = [ stats[i]['max_penetration'] for i in range(len(stats)) ]

fig = plt.figure()
ax = plt.subplot(111)
ax.set_title('Profiling data')
ax.set_xlabel('Step')
ax.set_ylabel('Value')
plt.grid(True)
ax.plot(number_of_overlaps, label='Overlaps', color=colors[0])
ax.plot(step_sizes, label='Stepsize', color=colors[1])
ax.plot(number_of_contact_points, label='Contacts', color=colors[2])
ax.plot(elastic_energy, label='Elastic Energy', color=colors[3])
ax.plot(kinetic_energy, label='Kinetic Energy', color=colors[4])
ax.plot(potential_energy, label='Potential Energy', color=colors[5])
ax.plot(penetrations, label='Penetrations', color=colors[6])
ax.legend()
plt.show()

NameError: name 'stats' is not defined